<h1>Part 1: Summary Statistics and Plots

<h3>Subpart 1: Profit Margins by Store

In [30]:
# Import data
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker

data = pd.read_csv("data.csv").set_index(["Location number", "Month"])
data.head()

State  Gross revenue   Fixed cost  Variable cost  \
Location number Month                                                    
1               1        TX    48628171.02   9218156.69    24278189.23   
                2        TX    48107179.24  10587747.35    25116844.41   
                3        TX    64352278.29  10193873.77    33703499.83   
                4        TX    45943677.96   8764165.00    24037719.47   
                5        TX    32761434.98   8022116.43    16432119.18   

                       Rental cost  Number of products  Owned  
Location number Month                                          
1               1           365880                4452  False  
                2           365880                4452  False  
                3           365880                4452  False  
                4           365880                4452  False  
                5           365880                4452  False

In [31]:
# Sum revenues and costs across months
data_annual = data.groupby(["Location number", "State", "Number of products", "Owned"]).sum()
data_annual = data_annual.reset_index().set_index("Location number")
data_annual.head()

,State,Number of products,Owned,Gross revenue,Fixed cost,Variable cost,Rental cost
Location number,,,,,,,
1,TX,4452,False,5.447389e+08,1.108956e+08,2.793687e+08,4390560
2,TX,4502,False,3.559730e+08,7.287395e+07,1.937291e+08,783240
3,IL,4620,True,3.317718e+08,6.429775e+07,1.848279e+08,0
4,GA,4481,False,2.639843e+08,6.410409e+07,1.432398e+08,1696920
5,NJ,4533,False,1.890016e+08,4.475050e+07,1.037553e+08,894000


Mean annual profit margin:  $95,453,449.83
Median annual profit margin:  $81,611,260.13



In [32]:
# Calculate and visualize annual profits, and find mean/median
data_annual["Profit"] = data_annual["Gross revenue"] - data_annual["Fixed cost"] - data_annual["Variable cost"] - data_annual["Rental cost"]
print("Mean annual profit margin: ", "${:,.2f}".format(data_annual["Profit"].mean()))
print("Median annual profit margin: ", "${:,.2f}".format(data_annual["Profit"].median()))
print()

fig, ax = plt.subplots()
ax.hist(data_annual["Profit"] / 10 ** 6, bins = range(0, 400, 25), edgecolor='black')
ax.set_title("Figure 1: Annual Profit Margins")
ax.set_xlabel("Profit Margin, Millions of Dollars")
ax.set_ylabel("Number of Stores")
plt.show()

While the distribution of annual profit margins does have a clear peak, it is heavily skewed right. This is also reflected by the fact that the mean is over 15% greater than the median. As such, the profit margins across locations don't look normally distributed. This is to be expected, since an unprofitable store will likely close (we see above that no store has a negative profit margin) but there is almost no limit on maximum possible profit.

<h3>Subpart 2: Aggregate Revenue

In [33]:
# Sum revenues and costs across stores
data_aggregate_monthly = data.groupby("Month").sum()
data_aggregate_monthly.head()

,Gross revenue,Fixed cost,Variable cost,Rental cost,Number of products,Owned
Month,,,,,,
1,9.304129e+09,2.069499e+09,4.943004e+09,49191170,1502156,36.0
2,9.786843e+09,1.964354e+09,5.180280e+09,49191170,1502156,36.0
3,1.158736e+10,2.296857e+09,6.126065e+09,49191170,1502156,36.0
4,9.105208e+09,1.915483e+09,4.754642e+09,49191170,1502156,36.0
5,9.347461e+09,1.877253e+09,4.860179e+09,49191170,1502156,36.0


In [34]:
# Combine months into quarters
data_aggregate_quarterly = data_aggregate_monthly.copy()
data_aggregate_quarterly.insert(0, "Quarter", [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4])
data_aggregate_quarterly = data_aggregate_quarterly.groupby("Quarter").sum()
data_aggregate_quarterly.head()

,Gross revenue,Fixed cost,Variable cost,Rental cost,Number of products,Owned
Quarter,,,,,,
1,3.067833e+10,6.330709e+09,1.624935e+10,147573510,4506468,108.0
2,2.975031e+10,6.168927e+09,1.546212e+10,147573510,4506468,108.0
3,2.988107e+10,6.241126e+09,1.536014e+10,147573510,4506468,108.0
4,2.993717e+10,6.227698e+09,1.583052e+10,147573510,4506468,108.0


In [35]:
# Visualize monthly and quarterly aggregate revenue
fig, (ax1, ax2) = plt.subplots(ncols = 2)
plt.suptitle("Figure 2: Aggregate Revenue")
ax1.set_ylabel("Revenue, Billions of Dollars")

ax1.bar(data_aggregate_monthly.index, data_aggregate_monthly["Gross revenue"] / 10 ** 9)
ax1.set_ylim(8, 12)
ax1.set_xlabel("Month")

ax2.bar(data_aggregate_quarterly.index, data_aggregate_quarterly["Gross revenue"] / 10 ** 9)
ax2.set_ylim(28, 32)
ax2.set_xlabel("Quarter")

plt.show()

<h1>Part 2: Data Cleaning

In [44]:
# Ensure that all owned stores are in states where there are also leased stores
data_leased = data[~data["Owned"]]
data_owned = data[data["Owned"]]
print(sorted(data_leased["State"].unique()))
print(sorted(data_owned["State"].unique()))

['CA', 'CO', 'GA', 'IL', 'NJ', 'TX', 'WA']
['CA', 'CO', 'GA', 'IL', 'NJ', 'TX', 'WA']


In [45]:
# Calculate mean monthly rental costs by state
data_leased = data_leased.groupby("State").mean()
data_leased.head()

,Gross revenue,Fixed cost,Variable cost,Rental cost,Number of products,Owned
State,,,,,,
CA,2.802657e+07,5.762085e+06,1.456200e+07,162564.000000,4490.216667,False
CO,3.174559e+07,6.740588e+06,1.663716e+07,159454.615385,4501.307692,False
GA,2.596567e+07,5.889000e+06,1.426121e+07,152734.000000,4496.460000,False
IL,2.780880e+07,5.862664e+06,1.466219e+07,159050.000000,4493.119048,False
NJ,2.657479e+07,6.129802e+06,1.481920e+07,165716.206897,4478.137931,False


In [46]:
# Impute rental costs for owned stores using the state mean
data_imputed = data.copy()
data_imputed["Rental cost"] = data_imputed.apply(
    lambda row: data_leased.loc[row["State"], "Rental cost"] if row["Owned"] else row["Rental cost"],
    axis=1
)
data_imputed.loc[data_imputed["Owned"]].head()

State  Gross revenue  Fixed cost  Variable cost  \
Location number Month                                                   
3               1        IL    30791018.69  6123108.00    17948338.14   
                2        IL    23805331.16  5014361.53    13051817.38   
                3        IL    33916624.99  6043273.71    19190769.99   
                4        IL    24170582.83  5196755.17    13505396.58   
                5        IL    24925332.00  4688566.82    14531349.44   

                       Rental cost  Number of products  Owned  
Location number Month                                          
3               1         159050.0                4620   True  
                2         159050.0                4620   True  
                3         159050.0                4620   True  
                4         159050.0                4620   True  
                5         159050.0                4620   True

In [47]:
# Sum revenues and costs across months again
data_imputed_annual = data_imputed.groupby(["Location number", "State", "Number of products", "Owned"]).sum()
data_imputed_annual = data_imputed_annual.reset_index().set_index("Location number")
data_imputed_annual.head()

,State,Number of products,Owned,Gross revenue,Fixed cost,Variable cost,Rental cost
Location number,,,,,,,
1,TX,4452,False,5.447389e+08,1.108956e+08,2.793687e+08,4390560.0
2,TX,4502,False,3.559730e+08,7.287395e+07,1.937291e+08,783240.0
3,IL,4620,True,3.317718e+08,6.429775e+07,1.848279e+08,1908600.0
4,GA,4481,False,2.639843e+08,6.410409e+07,1.432398e+08,1696920.0
5,NJ,4533,False,1.890016e+08,4.475050e+07,1.037553e+08,894000.0


In [49]:
# Recalculate projected rent-adjusted annual profits, and find mean/median
data_imputed_annual["Profit"] = data_imputed_annual["Gross revenue"] - data_imputed_annual["Fixed cost"] - data_imputed_annual["Variable cost"] - data_imputed_annual["Rental cost"]
print("Mean annual projected rent-adjusted profit margin: ", "${:,.2f}".format(data_imputed_annual["Profit"].mean()))
print("Median annual projected rent-adjusted profit margin: ", "${:,.2f}".format(data_imputed_annual["Profit"].median()))

Mean annual projected rent-adjusted profit margin:  $95,238,842.97
Median annual projected rent-adjusted profit margin:  $80,817,944.89
